<a href="https://colab.research.google.com/github/kejeon/in_dev_RN20Q/blob/main/train_resent20_q_deploy3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kejeon/in_dev_RN20Q.git
!pip install wandb -qU
%cd '/content/in_dev_RN20Q'

Cloning into 'in_dev_RN20Q'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 46 (delta 13), reused 39 (delta 10), pack-reused 0
Unpacking objects: 100% (46/46), 2.02 MiB | 8.26 MiB/s, done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
/content/in_dev_RN20Q


In [2]:
import wandb
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from model.resnet4c10q import ResNet20_Q
from model.resnet4c10 import resnet20
from mylib.KDResNetTrainer import ResNetTrainer

wandb.login(key='e0c11d3ff2bee4c8775ba05863038fdac671c043')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
api = wandb.Api()
artifact = api.artifact('jke1994/ResNet20/model:v179')
artifact.download(root='./pretrained_model')

state_dict = torch.load('./pretrained_model/ckpt.pth', map_location=torch.device('cpu'))

# update all keys in state_dict to remove module.
state_dict['net'] = {k.replace('module.', ''): v for k, v in state_dict['net'].items()}

teacher_model = resnet20()
# load pretrained model for resnet20
teacher_model.load_state_dict(state_dict['net'])

wandb:   1 of 1 files downloaded.  


<All keys matched successfully>

In [ ]:
lr = 0.01
arch_tag = "ResNet20Q"
batch_size = 128
dataset = "CIFAR10"

student_model = ResNet20_Q(a_bit=4, w_bit=1)

run = wandb.init(
    # set the wandb project where this run will be logged
    project="ResNet20_Q",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": arch_tag,
    "batch_size": batch_size,
    "dataset": dataset
    }
)

my_trainer = ResNetTrainer(dataset=dataset,
                           arch_tag=arch_tag,
                           student_model=student_model,
                           teacher_model=teacher_model,
                           device = 'cuda',
                           batch_size=batch_size,
                           lr = lr)

my_trainer.train_script(200)

wandb.finish()

runtime.unassign()

wandb: Currently logged in as: jke1994. Use `wandb login --relogin` to force relogin


100%|██████████| 170498071/170498071 [00:10<00:00, 16405680.10it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified

Epoch: 0


Epoch 0:   0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2919: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
Epoch 0: 100%|██████████| 391/391 [00:20<00:00, 19.08it/s, train_acc=0.173, xentropy=1.896]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:437: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


test_acc: 0.226
Saving..

Epoch: 1


...

Epoch 6: 100%|██████████| 391/391 [00:18<00:00, 21.23it/s, train_acc=0.404, xentropy=1.426]


test_acc: 0.410
Saving..

Epoch: 7


Epoch 7: 100%|██████████| 391/391 [00:18<00:00, 21.30it/s, train_acc=0.419, xentropy=1.387]


test_acc: 0.426
Saving..

Epoch: 8


Epoch 8: 100%|██████████| 391/391 [00:17<00:00, 21.76it/s, train_acc=0.435, xentropy=1.354]
